# Imports

In [1]:
from statistics import mode
import shutil
import os

# I - Création des tfrecord train et test

## 1 - paramètres :

In [2]:
path_images = '/tf/ship_data/train_v2'
path_to_csv = '/tf/ship_data/train_ship_segmentations_OD.csv'
image_rate = 0.01 # taux d'images de la base initiale, sachant qu'au maximum on peut prendre 42556 images avec bateau pour image_rate=1.
boat_rate = 0.8 # taux d'images contenant au moins un bateau
cut_rate = 0.7 # taux d'images (par rapport à nb_images) utilisées pour train. Le reste sera utilisé pour test.
tfrecord_dir = '/tf/annotations' # répertoire où train et test seront créés

## 2 - exécution du script :

Génère deux fichiers au format tfrecord dans le répertoire tfrecord_dir : deux fichiers train et test nommés selon train_25_80_70.tfrecord pour 25% des 42556 images de bateau, 80% d'images avec bateau réparties 70% dans train.

In [3]:
from generate_tfrecord import generate_tf_record
generate_tf_record(path_images, path_to_csv, image_rate, cut_rate, boat_rate, tfrecord_dir)

Création des dataframes train et test...
False


100%|██████████| 129/129 [00:01<00:00, 73.39it/s]


Création du tfrecord train...


100%|██████████| 296/296 [00:00<00:00, 799.20it/s]


Création du tfrecord test...


100%|██████████| 129/129 [00:00<00:00, 810.09it/s]

Tfrecords créés avec succès !
Enregistrés dans le répertoire :/tf/annotations


# II - Chargement du modèle

## 1 - paramètres :

Les modèles sont importés du "model zoo" de tensorflow : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [5]:
models_dir = '/tf/pretrained_models' # répertoire où les models seront téléchargés
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d2_coco17_tpu-32.tar.gz'

## 2 - exécution du script :

In [6]:
from download_model import download_model
download_model(model_url, models_dir)

353648640/353643040 [==============================] - 67s 0us/step


# III - Paramétrage des hyperparamètres du modèle

Le paramétrage suivant n'est pas exhaustif. Seuls les paramètres génériques, assurant à l'apprentissage puisse être lancé. D'autres hyperparamètres spécifiques au modèle choisi (generations des "anchor boxes" par exemple) doivent également être paramétrés.

##  1 - Lister des bases train et test disponibles, choix des paramètres

In [25]:
for path in [os.path.join(tfrecord_dir, p) for p in os.listdir(tfrecord_dir)]:
    print(path)

/tf/annotations/train_13250_90_70.tfrecord
/tf/annotations/test_13250_90_70.tfrecord
/tf/annotations/train_26500_90_70.tfrecord
/tf/annotations/test_26500_90_70.tfrecord
/tf/annotations/train_53000_90_70.tfrecord
/tf/annotations/test_53000_90_70.tfrecord


In [7]:
config_dict = {
    'num_classes' : 1,
    'batch_size' : 32,
    'train_tfrecord_path' : '/tf/annotations/train_13250_90_70.tfrecord',
    'test_tfrecord_path' : '/tf/annotations/test_13250_90_70.tfrecord',
    'label_map_path' : '/tf/ship_detect_tl/data/label_map.txt',
    'fine_tune_checkpoint' : '/tf/pretrained_models/checkpoints/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0',
    'fine_tune_checkpoint_type' : "detection"
}

## 2 -  Ecrire ces paramètres dans le fichier config

In [8]:
pipeline_config_path = '/tf/custom_models/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/pipeline.config'

In [9]:
from configure_pipeline import configure_pipeline
configure_pipeline(pipeline_config_path, config_dict)